# **<span style="color: #0098cd;">Comparación de algoritmos de Clustering (K-Means, DBSCAN, Jerárquico)</span>**

## **Objetivos**

## **Descripción**

El conjunto de datos con el que voy a trabajar se encuentra en: https://www.kaggle.com/datasets/umerrtx/machine-failure-prediction-using-sensor-data

Este conjunto de datos contiene datos de sensores recopilados de varias máquinas, con el objetivo de predecir fallos de las máquinas con antelación. Incluye una variedad de lecturas de sensores, así como las fallas de la máquina registradas.

Este archivo contiene 944 mediciones de sensores de diferentes máquinas, también contiene una etiqueta de si hay fallo o no.

Notas:
1. footfall: The number of people or objects passing by the machine.
2. tempMode: The temperature mode or setting of the machine.
3. AQ: Air quality index near the machine.
4. USS: Ultrasonic sensor data, indicating proximity measurements.
5. CS: Current sensor readings, indicating the electrical current usage of the machine.
6. VOC: Volatile organic compounds level detected near the machine.
7. RP: Rotational position or RPM (revolutions per minute) of the machine parts.
8. IP: Input pressure to the machine.
9. Temperature: The operating temperature of the machine.
10. fail: Binary indicator of machine failure (1 for failure, 0 for no failure).

## **Carga del dataset y análisis exploratorio**

#### Carga del dataset

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns


df = pd.read_csv('data.csv', sep=',') 

#### Número de instancias del dataset

#### Tipo de datos predominante en el dataset

#### Existencia de variables continuas y categóricas

#### Valores mínimo y máximo de la variable 'RP'

#### Para los algoritmos de clustering el dataset no puede tener instancias con datos faltantes. ¿Tiene este dataset datos faltantes?

#### Eliminación de la variable fail

## **<span style="color: #0098cd;">Algoritmo K-Means</span>**

## **<span style="color: #0098cd;">Algoritmo DBSCAN</span>**

## **<span style="color: #0098cd;">Algoritmo Clustering Jerárquico</span>**

## **Comparativa de algoritmos**